# Recommending Basics

## Recommending movies: retrieval

In [2]:
!pip install -q tensorflow-recommenders
!pip install -q scann

ERROR: Could not find a version that satisfies the requirement scann (from versions: none)
ERROR: No matching distribution found for scann
